In [1]:
!pip install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 24.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip install ragstack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.1

In [49]:
PROJECT_ID = "######"
LOCATION = "us-central1"

In [50]:
ASTRA_DB_API_ENDPOINT="######"
ASTRA_DB_APPLICATION_TOKEN="#######"

In [7]:
import getpass, os, requests

if "GCP_PROJECT_ID" not in os.environ:
  os.environ["GCP_PROJECT_ID"] = getpass.getpass("Provide your GCP Project ID")

if "LOCATION" not in os.environ:
  os.environ["LOCATION"] = getpass.getpass("Provide your GCP Location")

if "ASTRA_DB_ENDPOINT" not in os.environ:
  os.environ["ASTRA_DB_ENDPOINT"] = getpass.getpass("Provide your Astra DB Endpoint")

if "ASTRA_DB_TOKEN" not in os.environ:
  os.environ["ASTRA_DB_TOKEN"] = getpass.getpass("Provide your Astra DB Token")

Provide your GCP Project ID··········
Provide your GCP Location··········
Provide your Astra DB Endpoint··········
Provide your Astra DB Token··········


In [8]:
!gcloud config set project {os.getenv("GCP_PROJECT_ID")}

Updated property [core/project].


In [9]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
!gcloud auth list

In [11]:
# Define project information
PROJECT_ID=os.getenv("GCP_PROJECT_ID")
LOCATION=os.getenv("LOCATION")

In [13]:
# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [14]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part
)

In [15]:
model = GenerativeModel("gemini-1.0-pro")

In [16]:
prompt = """Create a numbered list of 10 items. Each item in the list should be a trend in the tech industry.

Each trend should be less than 5 words."""  # try your own prompt

In [17]:
responses = model.generate_content(prompt, stream=True)

In [18]:

for response in responses:
    print(response.text, end="")

## 10 Tech Trends for 2023:

1. Generative AI Everywhere
2. The Rise of the Metaverse
3. Quantum Computing Leaps Forward
4. 5G Gets Even Faster
5. Edge Computing Takes Center Stage
6. Biometric Security Goes Mainstream
7. Hyperautomation Drives Efficiency
8. Low-Code Revolutionizes Development
9. Sustainable Tech Takes Priority
10. The AI Talent Gap Widens

In [19]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

In [20]:
responses = model.generate_content(
    "Why is the sky blue?",
    generation_config=generation_config,
    stream=True,
)

In [21]:
for response in responses:
    print(response.text, end="")

The sky appears blue because of a phenomenon called Rayleigh scattering. Sunlight is composed of all the colors of the rainbow, each with a different wavelength. When sunlight enters the Earth's atmosphere, it interacts with the tiny molecules of gas that make up the air. These molecules scatter the light in all directions, but they scatter shorter wavelengths (like blue) more effectively than longer wavelengths (like red). This means that blue light is scattered more in all directions, making the sky appear blue to our eyes.

Here's a more detailed explanation:

1. **Sunlight is white:** Sunlight is actually a mixture of all the colors of the rainbow, each with a different wavelength. You can see this yourself by shining a beam of sunlight through a prism, which will separate the light into its different colors.
2. **The atmosphere scatters light:** When sunlight enters the Earth's atmosphere, it interacts with the tiny molecules of gas that make up the air. These molecules scatter th

In [22]:
source_img_data = requests.get('https://drive.google.com/uc?export=view&id=15ddcn-AIxpvRdWcFGvIr77XLWdo4Maof').content

In [23]:
with open('coffee_maker_part.png', 'wb') as handler:
  handler.write(source_img_data)

In [24]:
from langchain_google_vertexai import ChatVertexAI

In [25]:
from langchain.schema.messages import HumanMessage
from PIL import Image, ImageFile
import os, sys

In [26]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision",safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },)

In [27]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "coffee_maker_part.png"},
}
text_message = {
    "type": "text",
    "text": "What is this image? Share a link to purchase a replacement",
}

In [28]:
message = HumanMessage(content=[text_message, image_message])

In [29]:
output = chat([message])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [30]:
print(output.content)

 This is a replacement part for a coffee machine. It is a filter basket that holds the coffee grounds. You can purchase a replacement part online. Here is a link to a replacement part:

https://www.amazon.com/Replacement-Filter-Basket-Breville-Machines/dp/B07578832P


In [31]:
import pandas as pd

d = {'name': ["Saucer", "Saucer Ceramic", "Milk Jug Assembly", "Handle Steam Wand Kit (New Version From 0735 PDC)", "Spout Juice Small (From 0637 to 1041 PDC)", "Cleaning Steam Wand", "Jug Frothing", "Spoon Tamping 50mm", "Collar Grouphead 50mm", "Filter 2 Cup Dual Wall 50mm", "Filter 1 Cup 50mm", "Water Tank Assembly", "Portafilter Assembly 50mm", "Milk Jug Assembly", "Filter 2 Cup 50mm" ],
     'url': ["https://www.breville.com/us/en/parts-accessories/parts/sp0014946.html?sku=SP0014946", "https://www.breville.com/us/en/parts-accessories/parts/sp0014914.html?sku=SP0014914", "https://www.breville.com/us/en/parts-accessories/parts/sp0011391.html?sku=SP0011391", "https://www.breville.com/us/en/parts-accessories/parts/sp0010719.html?sku=SP0010719", "https://www.breville.com/us/en/parts-accessories/parts/sp0010718.html?sku=SP0010718", "https://www.breville.com/us/en/parts-accessories/parts/sp0003247.html?sku=SP0003247", "https://www.breville.com/us/en/parts-accessories/parts/sp0003246.html?sku=SP0003246", "https://www.breville.com/us/en/parts-accessories/parts/sp0003243.html?sku=SP0003243", "https://www.breville.com/us/en/parts-accessories/parts/sp0003232.html?sku=SP0003232", "https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231", "https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230", "https://www.breville.com/us/en/parts-accessories/parts/sp0003225.html?sku=SP0003225", "https://www.breville.com/us/en/parts-accessories/parts/sp0003216.html?sku=SP0003216", "https://www.breville.com/us/en/parts-accessories/parts/sp0001875.html?sku=SP0001875", "https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166"],
     'price': ["10.95", "4.99", "14.95", "8.95", "10.95", "6.95", "24.95", "8.95", "6.95", "12.95", "12.95", "14.95", "10.95", "16.95", "11.95"],
     'image': ["https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014946/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0014914/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0011391/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010719/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0010718/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003247/tile.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/BES250/SP0003246/SP0003246_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003243/SP0003243_IMAGE1_400X400.jpg", "https://assets.breville.com/cdn-cgi/image/width=400,format=auto/Spare+Parts+/Espresso+Machines/ESP8/SP0003232/SP0003232_IMAGE1_400x400.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003225/tile.jpg", "https://www.breville.com/content/dam/breville/ca/catalog/products/images/sp0/sp0003216/tile.jpg", "https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0001875/tile.jpg", "https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg"]}
df = pd.DataFrame(data=d)
df

,name,url,price,image
0,Saucer,https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
1,Saucer Ceramic,https://www.breville.com/us/en/parts-accessori...,4.99,https://www.breville.com/content/dam/breville/...
2,Milk Jug Assembly,https://www.breville.com/us/en/parts-accessori...,14.95,https://www.breville.com/content/dam/breville/...
3,Handle Steam Wand Kit (New Version From 0735 PDC),https://www.breville.com/us/en/parts-accessori...,8.95,https://www.breville.com/content/dam/breville/...
4,Spout Juice Small (From 0637 to 1041 PDC),https://www.breville.com/us/en/parts-accessori...,10.95,https://www.breville.com/content/dam/breville/...
5,Cleaning Steam Wand,https://www.breville.com/us/en/parts-accessori...,6.95,https://www.breville.com/content/dam/breville/...
6,Jug Frothing,https://www.breville.com/us/en/parts-accessori...,24.95,https://assets.breville.com/cdn-cgi/image/widt...
7,Spoon Tamping 50mm,https://www.breville.com/us/en/parts-accessori...,8.95,https://assets.breville.com/cdn-cgi/image/widt...
8,Collar Grouphead 50mm,https://www.breville.com/us/en/parts-accessori...,6.95,https://assets.breville.com/cdn-cgi/image/widt...
9,Filter 2 Cup Dual Wall 50mm,https://www.breville.com/us/en/parts-accessori...,12.95,https://www.breville.com/content/dam/breville/...


In [32]:
import vertexai, json, requests
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
from astrapy.db import AstraDB, AstraDBCollection
from google.colab import files

In [33]:
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

In [34]:
# Initialize our vector db
astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"), api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))

In [35]:
collection = astra_db.create_collection(collection_name="coffee_shop_ecommerce", dimension=1408)

In [36]:
for i in range(len(df)):
  name = df.loc[i, "name"]
  image = df.loc[i, "image"]
  price = df.loc[i, "price"]
  url = df.loc[i, "url"]

  # Download this product's image and save it to the Colab filesystem.
  # In a production system this binary data would be stored in Google Cloud Storage
  img_data = requests.get(image).content
  with open(f'{name}.png', 'wb') as handler:
    handler.write(img_data)

  # load the image from filesystem and compute the embedding value
  img = Image.load_from_file(f'{name}.png')
  embeddings = model.get_embeddings(image=img, contextual_text=name)

  try:
    # add to the AstraDB Vector Database
    collection.insert_one({
        "_id": i,
        "name": name,
        "image": image,
        "url": url,
        "price": price,
        "$vector": embeddings.image_embedding,
      })
  except Exception as error:
    # if you've already added this record, skip the error message
    error_info = json.loads(str(error))
    if error_info[0]['errorCode'] == "DOCUMENT_ALREADY_EXISTS":
      print("Document already exists in the database.  Skipping.")

In [37]:
import json

# Embed the similar item
img = Image.load_from_file('coffee_maker_part.png')

In [38]:
embeddings = model.get_embeddings(image=img, contextual_text="A espresso machine part")

In [39]:
embeddings.image_embedding

[0.0103441058,
 -0.0423361734,
 0.00252593122,
 0.0468367673,
 -0.0058448785,
 -0.0222764723,
 -0.0102796685,
 -0.0213931967,
 -0.0253920723,
 -0.00449404586,
 -0.0144733302,
 -0.0385555103,
 0.0178168286,
 0.161146164,
 -0.0240376759,
 0.0039360458,
 0.056310948,
 -0.00262508495,
 -0.0355716608,
 0.0185565744,
 -0.0134995198,
 -0.009202688,
 -0.00511704618,
 -0.0111004887,
 0.0024302043,
 0.00353298197,
 0.00684152637,
 -0.017458545,
 0.0498935245,
 -0.0408515334,
 0.0270688497,
 -0.00324034202,
 0.00235821214,
 0.017361708,
 0.0509899594,
 -0.0192034747,
 -0.0190861151,
 0.0200850032,
 -0.00735367695,
 -0.00653035706,
 -0.0258878134,
 0.0159181897,
 -0.028727999,
 -0.0242899694,
 0.0104888212,
 -0.0224466659,
 -0.0308991913,
 0.00114864146,
 0.00913378876,
 -0.000931869203,
 -0.0330106653,
 -0.0142502971,
 0.0255831,
 -0.0319835171,
 0.0354432389,
 0.00188285636,
 -0.0174728911,
 -0.0277421549,
 0.0410693772,
 -0.0386418737,
 0.00900237635,
 -0.00827283878,
 0.0344877727,
 -0.0153780

In [40]:
# Perform the vector search against AstraDB Vector
documents = collection.vector_find(
    embeddings.image_embedding,
    limit=3,
)

In [41]:
documents

[{'_id': 14,
  'name': 'Filter 2 Cup 50mm',
  'image': 'https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166',
  'price': '11.95',
  '$similarity': 0.9142839},
 {'_id': 9,
  'name': 'Filter 2 Cup Dual Wall 50mm',
  'image': 'https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231',
  'price': '12.95',
  '$similarity': 0.9142827},
 {'_id': 10,
  'name': 'Filter 1 Cup 50mm',
  'image': 'https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg',
  'url': 'https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230',
  'price': '12.95',
  '$similarity': 0.9116894}]

In [42]:
related_products_csv = "name, image, price, url\n"
for doc in documents:
  related_products_csv += f"{doc['name']}, {doc['image']}, {doc['price']}, {doc['url']},\n"

In [43]:
print(related_products_csv)

name, image, price, url
Filter 2 Cup 50mm, https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg, 11.95, https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166,
Filter 2 Cup Dual Wall 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231,
Filter 1 Cup 50mm, https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg, 12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230,



In [44]:
image_message = {
    "type": "image_url",
    "image_url": {"url": "/content/coffee_maker_part.png"},
}
text_message = {
    "type": "text",
    "text": f"What we have in this image? Share the URL and price to purchase a replacement. Here are related products {related_products_csv}",
}

In [45]:
message = HumanMessage(content=[text_message, image_message])

In [46]:
chat = ChatVertexAI(model_name="gemini-1.0-pro-vision",safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
    },)

In [47]:
output = chat([message])

In [48]:
print(output.content)

 This is a filter basket for a Breville espresso machine. You can purchase a replacement on the Breville website for $12.95. The URL for the product page is https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230.

Here are some related products that you may also be interested in:
Filter 2 Cup 50mm: https://www.breville.com/content/dam/breville/us/catalog/products/images/sp0/sp0000166/tile.jpg, $11.95, https://www.breville.com/us/en/parts-accessories/parts/sp0000166.html?sku=SP0000166
Filter 2 Cup Dual Wall 50mm: https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003231/tile.jpg, $12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003231.html?sku=SP0003231
Filter 1 Cup 50mm: https://www.breville.com/content/dam/breville/au/catalog/products/images/sp0/sp0003230/tile.jpg, $12.95, https://www.breville.com/us/en/parts-accessories/parts/sp0003230.html?sku=SP0003230
